# Detecção de Ataques de Rede em Tempo Real com DistilBERT Otimizado

Este notebook demonstra como adaptar o DistilBERT para detecção de ataques DDoS em tempo real usando dados de rede tabulares.

**Objetivo**: Criar um modelo leve e eficiente para rodar em Raspberry Pi detectando ataques em tempo real.

## 0. Instalação de Dependências

In [ ]:
# Verificar se estamos no Google Colab
try:
    import google.colab
    IN_COLAB = True
    from google.colab import files
except:
    IN_COLAB = False

print(f"Executando no Google Colab: {IN_COLAB}")

# Instalar dependências
!pip install -q transformers torch onnx onnxruntime numpy pandas scikit-learn matplotlib seaborn optimum[onnxruntime]

In [ ]:
# Importar bibliotecas
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertConfig, Trainer, TrainingArguments
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import onnx
import onnxruntime as ort
import time
import warnings
warnings.filterwarnings('ignore')

# Configurar device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando device: {device}")

## 1. Upload e Análise dos Dados

In [ ]:
if IN_COLAB:
    print("Faça upload dos seus arquivos CSV de dados de rede:")
    uploaded = files.upload()
    
    # Listar arquivos carregados
    csv_files = [f for f in uploaded.keys() if f.endswith('.csv')]
    print(f"Arquivos CSV carregados: {csv_files}")
else:
    # Para execução local, listar arquivos CSV no diretório
    import glob
    csv_files = glob.glob("*.csv")
    print(f"Arquivos CSV encontrados: {csv_files}")

In [ ]:
def load_and_analyze_data(csv_files, sample_size=50000):
    """Carregar e analisar dados de múltiplos arquivos CSV"""
    
    all_data = []
    
    for file in csv_files[:5]:  # Limitar a 5 arquivos para teste
        print(f"Carregando {file}...")
        try:
            df = pd.read_csv(file)
            # Amostrar dados para reduzir tempo de processamento
            if len(df) > sample_size:
                df = df.sample(n=sample_size, random_state=42)
            all_data.append(df)
            print(f"  - Shape: {df.shape}")
            print(f"  - Labels únicos: {df['label'].unique()}")
        except Exception as e:
            print(f"  - Erro ao carregar {file}: {e}")
    
    # Combinar todos os dados
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\nDados combinados: {combined_df.shape}")
        return combined_df
    else:
        print("Nenhum dado foi carregado com sucesso.")
        return None

# Carregar dados
df = load_and_analyze_data(csv_files)

if df is not None:
    print("\n=== ANÁLISE DOS DADOS ===")
    print(f"Shape total: {df.shape}")
    print(f"\nColunas: {list(df.columns)}")
    print(f"\nDistribuição de labels:")
    print(df['label'].value_counts())
    print(f"\nValores nulos por coluna:")
    print(df.isnull().sum().sum())

## 2. Pré-processamento dos Dados

In [ ]:
def preprocess_data(df):
    """Pré-processar dados para o modelo"""
    
    # Remover colunas com muitos valores nulos ou constantes
    df_clean = df.copy()
    
    # Remover colunas com mais de 50% de valores nulos
    null_threshold = 0.5
    null_cols = df_clean.columns[df_clean.isnull().mean() > null_threshold]
    df_clean = df_clean.drop(columns=null_cols)
    print(f"Removidas {len(null_cols)} colunas com muitos valores nulos")
    
    # Preencher valores nulos restantes
    df_clean = df_clean.fillna(0)
    
    # Separar features e labels
    X = df_clean.drop('label', axis=1)
    y = df_clean['label']
    
    # Codificar labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Normalizar features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(f"\nFeatures shape: {X_scaled.shape}")
    print(f"Labels shape: {y_encoded.shape}")
    print(f"Número de classes: {len(label_encoder.classes_)}")
    print(f"Classes: {label_encoder.classes_}")
    
    return X_scaled, y_encoded, label_encoder, scaler, list(X.columns)

# Pré-processar dados
if df is not None:
    X, y, label_encoder, scaler, feature_names = preprocess_data(df)
    
    # Dividir em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\nTreino: {X_train.shape}, Teste: {X_test.shape}")

## 3. Modelo DistilBERT Adaptado para Dados Tabulares

In [ ]:
class TabularDistilBERT(nn.Module):
    """DistilBERT adaptado para dados tabulares de rede"""
    
    def __init__(self, num_features, num_classes, hidden_dim=256):
        super().__init__()
        
        # Configuração reduzida do DistilBERT para eficiência
        config = DistilBertConfig(
            vocab_size=1000,  # Reduzido
            max_position_embeddings=512,
            n_layers=3,  # Reduzido de 6 para 3
            n_heads=8,
            dim=hidden_dim,  # Reduzido
            hidden_dim=hidden_dim * 4,
            dropout=0.1,
            attention_dropout=0.1
        )
        
        # Camada de projeção para converter features tabulares em embeddings
        self.feature_projection = nn.Linear(num_features, hidden_dim)
        
        # DistilBERT backbone (sem embeddings de palavras)
        self.distilbert = DistilBertModel(config)
        
        # Cabeça de classificação
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
        self.num_features = num_features
        self.num_classes = num_classes
    
    def forward(self, features):
        # Projetar features para dimensão do modelo
        batch_size = features.shape[0]
        
        # Converter features em embeddings
        embeddings = self.feature_projection(features)  # [batch_size, hidden_dim]
        
        # Adicionar dimensão de sequência (simular tokens)
        embeddings = embeddings.unsqueeze(1)  # [batch_size, 1, hidden_dim]
        
        # Criar attention mask
        attention_mask = torch.ones(batch_size, 1, device=features.device)
        
        # Passar pelo DistilBERT
        outputs = self.distilbert(
            inputs_embeds=embeddings,
            attention_mask=attention_mask
        )
        
        # Usar o último hidden state
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_dim]
        
        # Classificação
        logits = self.classifier(pooled_output)
        
        return logits

# Criar modelo
if df is not None:
    num_features = X.shape[1]
    num_classes = len(label_encoder.classes_)
    
    model = TabularDistilBERT(num_features, num_classes, hidden_dim=128)
    model = model.to(device)
    
    print(f"Modelo criado:")
    print(f"  - Features de entrada: {num_features}")
    print(f"  - Classes de saída: {num_classes}")
    print(f"  - Parâmetros totais: {sum(p.numel() for p in model.parameters()):,}")

## 4. Dataset e DataLoader

In [ ]:
class NetworkDataset(Dataset):
    """Dataset para dados de rede"""
    
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return {
            'features': self.features[idx],
            'labels': self.labels[idx]
        }

# Criar datasets
if df is not None:
    train_dataset = NetworkDataset(X_train, y_train)
    test_dataset = NetworkDataset(X_test, y_test)
    
    # Criar dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    print(f"Dataset de treino: {len(train_dataset)} amostras")
    print(f"Dataset de teste: {len(test_dataset)} amostras")

## 5. Treinamento do Modelo

In [ ]:
def train_model(model, train_loader, test_loader, num_epochs=5):
    """Treinar o modelo"""
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    
    train_losses = []
    train_accuracies = []
    
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        for batch in train_loader:
            features = batch['features'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = total_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        
        print(f"Época {epoch+1}/{num_epochs}:")
        print(f"  Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
        
        # Avaliar no conjunto de teste
        if (epoch + 1) % 2 == 0:
            test_acc = evaluate_model(model, test_loader)
            print(f"  Test Accuracy: {test_acc:.2f}%")
    
    return train_losses, train_accuracies

def evaluate_model(model, test_loader):
    """Avaliar o modelo"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            features = batch['features'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    model.train()
    return 100 * correct / total

# Treinar modelo
if df is not None:
    print("Iniciando treinamento...")
    train_losses, train_accuracies = train_model(model, train_loader, test_loader, num_epochs=3)
    
    # Avaliação final
    final_accuracy = evaluate_model(model, test_loader)
    print(f"\nAccuracy final no teste: {final_accuracy:.2f}%")

## 6. Análise Detalhada de Performance

In [ ]:
def detailed_evaluation(model, test_loader, label_encoder):
    """Avaliação detalhada com métricas"""
    
    model.eval()
    all_predictions = []
    all_labels = []
    inference_times = []
    
    with torch.no_grad():
        for batch in test_loader:
            features = batch['features'].to(device)
            labels = batch['labels'].to(device)
            
            # Medir tempo de inferência
            start_time = time.time()
            outputs = model(features)
            inference_time = (time.time() - start_time) * 1000  # ms
            inference_times.append(inference_time)
            
            _, predicted = torch.max(outputs.data, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calcular métricas
    accuracy = accuracy_score(all_labels, all_predictions)
    
    print("\n=== RELATÓRIO DE PERFORMANCE ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Tempo médio de inferência: {np.mean(inference_times):.2f} ms")
    print(f"Throughput: {1000/np.mean(inference_times):.2f} predições/segundo")
    
    # Relatório de classificação
    print("\n=== RELATÓRIO DE CLASSIFICAÇÃO ===")
    print(classification_report(
        all_labels, all_predictions, 
        target_names=label_encoder.classes_,
        digits=4
    ))
    
    # Matriz de confusão
    cm = confusion_matrix(all_labels, all_predictions)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.title('Matriz de Confusão')
    plt.ylabel('Verdadeiro')
    plt.xlabel('Predito')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    return accuracy, np.mean(inference_times)

# Avaliação detalhada
if df is not None:
    accuracy, avg_inference_time = detailed_evaluation(model, test_loader, label_encoder)

## 7. Otimização para Raspberry Pi

In [ ]:
class OptimizedNetworkDetector(nn.Module):
    """Wrapper otimizado para exportação ONNX"""
    
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, features):
        logits = self.model(features)
        probabilities = torch.softmax(logits, dim=-1)
        return logits, probabilities

def export_optimized_model(model, X_sample, scaler, label_encoder, feature_names):
    """Exportar modelo otimizado para ONNX"""
    
    # Criar wrapper otimizado
    optimized_model = OptimizedNetworkDetector(model)
    optimized_model.eval()
    
    # Preparar input de exemplo
    dummy_input = torch.FloatTensor(X_sample[:1]).to(device)
    
    # Exportar para ONNX
    torch.onnx.export(
        optimized_model,
        dummy_input,
        'network_attack_detector.onnx',
        input_names=['features'],
        output_names=['logits', 'probabilities'],
        dynamic_axes={
            'features': {0: 'batch_size'},
            'logits': {0: 'batch_size'},
            'probabilities': {0: 'batch_size'}
        },
        opset_version=14,
        do_constant_folding=True
    )
    
    print("Modelo exportado para ONNX: network_attack_detector.onnx")
    
    # Quantizar modelo
    from onnxruntime.quantization import quantize_dynamic, QuantType
    
    quantize_dynamic(
        'network_attack_detector.onnx',
        'network_attack_detector_quantized.onnx',
        weight_type=QuantType.QInt8
    )
    
    print("Modelo quantizado: network_attack_detector_quantized.onnx")
    
    # Salvar metadados
    import pickle
    
    metadata = {
        'scaler': scaler,
        'label_encoder': label_encoder,
        'feature_names': feature_names,
        'num_features': len(feature_names),
        'num_classes': len(label_encoder.classes_),
        'classes': label_encoder.classes_.tolist()
    }
    
    with open('model_metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)
    
    print("Metadados salvos: model_metadata.pkl")
    
    return metadata

# Exportar modelo otimizado
if df is not None:
    print("Exportando modelo otimizado...")
    metadata = export_optimized_model(model, X_test, scaler, label_encoder, feature_names)
    
    # Verificar tamanhos dos arquivos
    import os
    original_size = os.path.getsize('network_attack_detector.onnx') / (1024*1024)
    quantized_size = os.path.getsize('network_attack_detector_quantized.onnx') / (1024*1024)
    
    print(f"\nTamanho do modelo original: {original_size:.2f} MB")
    print(f"Tamanho do modelo quantizado: {quantized_size:.2f} MB")
    print(f"Redução: {(1 - quantized_size/original_size)*100:.1f}%")

## 8. Sistema de Monitoramento em Tempo Real para Raspberry Pi

In [ ]:
# Criar script de monitoramento em tempo real
monitoring_script = '''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Sistema de Detecção de Ataques de Rede em Tempo Real
Otimizado para Raspberry Pi
"""

import onnxruntime as ort
import numpy as np
import pandas as pd
import pickle
import time
import argparse
import json
from datetime import datetime
import threading
import queue
import sys

class NetworkAttackDetector:
    def __init__(self, model_path, metadata_path):
        """Inicializar detector de ataques"""
        
        print("Carregando modelo...")
        self.session = ort.InferenceSession(model_path)
        
        print("Carregando metadados...")
        with open(metadata_path, 'rb') as f:
            self.metadata = pickle.load(f)
        
        self.scaler = self.metadata['scaler']
        self.label_encoder = self.metadata['label_encoder']
        self.feature_names = self.metadata['feature_names']
        self.classes = self.metadata['classes']
        
        print(f"Modelo carregado com sucesso!")
        print(f"Classes detectáveis: {self.classes}")
        
        # Estatísticas
        self.total_predictions = 0
        self.attack_detections = 0
        self.inference_times = []
    
    def preprocess_features(self, features_dict):
        """Pré-processar features de entrada"""
        
        # Converter para array na ordem correta
        features_array = np.array([features_dict.get(name, 0.0) for name in self.feature_names])
        
        # Normalizar
        features_scaled = self.scaler.transform(features_array.reshape(1, -1))
        
        return features_scaled.astype(np.float32)
    
    def predict(self, features_dict):
        """Fazer predição de ataque"""
        
        # Pré-processar
        features = self.preprocess_features(features_dict)
        
        # Inferência
        start_time = time.time()
        ort_inputs = {'features': features}
        logits, probabilities = self.session.run(None, ort_inputs)
        inference_time = (time.time() - start_time) * 1000
        
        # Processar resultado
        predicted_class_idx = np.argmax(probabilities[0])
        predicted_class = self.classes[predicted_class_idx]
        confidence = probabilities[0][predicted_class_idx]
        
        # Atualizar estatísticas
        self.total_predictions += 1
        self.inference_times.append(inference_time)
        
        if predicted_class != 'Benign':  # Assumindo que 'Benign' é tráfego normal
            self.attack_detections += 1
        
        return {
            'timestamp': datetime.now().isoformat(),
            'predicted_class': predicted_class,
            'confidence': float(confidence),
            'is_attack': predicted_class != 'Benign',
            'inference_time_ms': inference_time,
            'all_probabilities': probabilities[0].tolist()
        }
    
    def get_statistics(self):
        """Obter estatísticas do detector"""
        
        if not self.inference_times:
            return {}
        
        return {
            'total_predictions': self.total_predictions,
            'attack_detections': self.attack_detections,
            'attack_rate': self.attack_detections / self.total_predictions if self.total_predictions > 0 else 0,
            'avg_inference_time_ms': np.mean(self.inference_times),
            'max_inference_time_ms': np.max(self.inference_times),
            'min_inference_time_ms': np.min(self.inference_times),
            'throughput_per_second': 1000 / np.mean(self.inference_times)
        }

class RealTimeMonitor:
    def __init__(self, detector, log_file='attack_log.json'):
        self.detector = detector
        self.log_file = log_file
        self.data_queue = queue.Queue()
        self.running = False
    
    def log_detection(self, result):
        """Registrar detecção em arquivo"""
        
        with open(self.log_file, 'a') as f:
            f.write(json.dumps(result) + '\n')
    
    def process_data_stream(self):
        """Processar stream de dados em tempo real"""
        
        while self.running:
            try:
                # Obter dados da fila
                features_dict = self.data_queue.get(timeout=1)
                
                # Fazer predição
                result = self.detector.predict(features_dict)
                
                # Log se for ataque
                if result['is_attack']:
                    print(f"🚨 ATAQUE DETECTADO: {result['predicted_class']} (Confiança: {result['confidence']:.3f})")
                    self.log_detection(result)
                else:
                    print(f"✅ Tráfego normal (Confiança: {result['confidence']:.3f})")
                
                self.data_queue.task_done()
                
            except queue.Empty:
                continue
            except Exception as e:
                print(f"Erro no processamento: {e}")
    
    def start_monitoring(self):
        """Iniciar monitoramento"""
        
        self.running = True
        monitor_thread = threading.Thread(target=self.process_data_stream)
        monitor_thread.daemon = True
        monitor_thread.start()
        
        print("Monitoramento iniciado...")
        return monitor_thread
    
    def stop_monitoring(self):
        """Parar monitoramento"""
        self.running = False
    
    def add_data(self, features_dict):
        """Adicionar dados para análise"""
        self.data_queue.put(features_dict)

def simulate_network_data(csv_file, detector, monitor, delay=1.0):
    """Simular dados de rede em tempo real"""
    
    print(f"Carregando dados de simulação: {csv_file}")
    df = pd.read_csv(csv_file)
    
    print(f"Iniciando simulação com {len(df)} amostras...")
    
    for idx, row in df.iterrows():
        # Converter linha para dicionário (excluindo label)
        features_dict = row.drop('label').to_dict()
        
        # Adicionar à fila de monitoramento
        monitor.add_data(features_dict)
        
        # Mostrar progresso
        if (idx + 1) % 100 == 0:
            stats = detector.get_statistics()
            print(f"\nProcessadas {idx + 1} amostras")
            print(f"Taxa de ataques: {stats.get('attack_rate', 0):.3f}")
            print(f"Tempo médio: {stats.get('avg_inference_time_ms', 0):.2f} ms")
        
        time.sleep(delay)

def main():
    parser = argparse.ArgumentParser(description='Detector de Ataques de Rede em Tempo Real')
    parser.add_argument('--model', default='network_attack_detector_quantized.onnx', help='Modelo ONNX')
    parser.add_argument('--metadata', default='model_metadata.pkl', help='Metadados do modelo')
    parser.add_argument('--simulate', type=str, help='Arquivo CSV para simulação')
    parser.add_argument('--delay', type=float, default=0.1, help='Delay entre amostras (segundos)')
    parser.add_argument('--interactive', action='store_true', help='Modo interativo')
    parser.add_argument('--benchmark', action='store_true', help='Benchmark de performance')
    
    args = parser.parse_args()
    
    # Inicializar detector
    try:
        detector = NetworkAttackDetector(args.model, args.metadata)
        monitor = RealTimeMonitor(detector)
    except Exception as e:
        print(f"Erro ao inicializar detector: {e}")
        sys.exit(1)
    
    if args.benchmark:
        # Benchmark de performance
        print("Executando benchmark...")
        
        # Criar dados de teste
        test_features = {name: np.random.randn() for name in detector.feature_names}
        
        # Executar múltiplas predições
        for i in range(1000):
            detector.predict(test_features)
        
        stats = detector.get_statistics()
        print(f"\nResultados do benchmark:")
        print(f"Predições: {stats['total_predictions']}")
        print(f"Tempo médio: {stats['avg_inference_time_ms']:.2f} ms")
        print(f"Throughput: {stats['throughput_per_second']:.2f} predições/segundo")
        
    elif args.simulate:
        # Simular dados de rede
        monitor_thread = monitor.start_monitoring()
        
        try:
            simulate_network_data(args.simulate, detector, monitor, args.delay)
        except KeyboardInterrupt:
            print("\nInterrompido pelo usuário")
        finally:
            monitor.stop_monitoring()
            
            # Mostrar estatísticas finais
            stats = detector.get_statistics()
            print(f"\n=== ESTATÍSTICAS FINAIS ===")
            for key, value in stats.items():
                print(f"{key}: {value}")
    
    elif args.interactive:
        # Modo interativo
        print("\nModo interativo ativado.")
        print("Digite valores para as features ou 'sair' para encerrar.")
        print(f"Features necessárias: {detector.feature_names[:5]}... (total: {len(detector.feature_names)})")
        
        while True:
            try:
                # Exemplo simples: usar valores aleatórios
                input("Pressione Enter para gerar predição com dados aleatórios (ou Ctrl+C para sair): ")
                
                test_features = {name: np.random.randn() for name in detector.feature_names}
                result = detector.predict(test_features)
                
                print(f"\nResultado:")
                print(f"  Classe: {result['predicted_class']}")
                print(f"  Confiança: {result['confidence']:.3f}")
                print(f"  É ataque: {result['is_attack']}")
                print(f"  Tempo: {result['inference_time_ms']:.2f} ms")
                
            except KeyboardInterrupt:
                break
    
    else:
        print("Use --simulate, --interactive ou --benchmark")
        parser.print_help()

if __name__ == '__main__':
    main()
'''

# Salvar script
with open('realtime_network_monitor.py', 'w', encoding='utf-8') as f:
    f.write(monitoring_script)

print("Script de monitoramento criado: realtime_network_monitor.py")

## 9. Arquivos de Configuração e Documentação

In [ ]:
# Criar requirements.txt
requirements = '''onnxruntime==1.15.1
numpy==1.24.3
pandas==2.0.3
scikit-learn==1.3.0
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

# Criar script de instalação
install_script = '''#!/bin/bash
# Script de instalação para Raspberry Pi

echo "Instalando dependências para detecção de ataques de rede..."

# Atualizar sistema
sudo apt update
sudo apt upgrade -y

# Instalar Python e pip
sudo apt install python3 python3-pip -y

# Instalar dependências Python
pip3 install -r requirements.txt

# Criar diretório de logs
mkdir -p logs

echo "Instalação concluída!"
echo "Para testar: python3 realtime_network_monitor.py --benchmark"
'''

with open('install.sh', 'w') as f:
    f.write(install_script)

# Criar README detalhado
readme = f'''# Sistema de Detecção de Ataques de Rede em Tempo Real

Sistema baseado em DistilBERT otimizado para detectar ataques DDoS em tempo real em dispositivos IoT como Raspberry Pi.

## Características do Modelo

- **Modelo**: DistilBERT adaptado para dados tabulares
- **Otimizações**: Quantização INT8, ONNX Runtime
- **Tamanho**: ~{quantized_size:.1f} MB (redução de {(1 - quantized_size/original_size)*100:.1f}%)
- **Performance**: ~{avg_inference_time:.1f} ms por predição
- **Classes detectáveis**: {len(label_encoder.classes_)} tipos de ataques

## Tipos de Ataques Detectados

{chr(10).join([f"- {cls}" for cls in label_encoder.classes_])}

## Instalação no Raspberry Pi

1. Transfira todos os arquivos para o Raspberry Pi
2. Execute o script de instalação:
```bash
chmod +x install.sh
./install.sh
```

## Uso

### Benchmark de Performance
```bash
python3 realtime_network_monitor.py --benchmark
```

### Simulação com Dados CSV
```bash
python3 realtime_network_monitor.py --simulate dados_rede.csv --delay 0.1
```

### Modo Interativo
```bash
python3 realtime_network_monitor.py --interactive
```

## Arquivos Incluídos

- `network_attack_detector_quantized.onnx`: Modelo otimizado
- `model_metadata.pkl`: Metadados (scaler, encoder, features)
- `realtime_network_monitor.py`: Sistema de monitoramento
- `requirements.txt`: Dependências Python
- `install.sh`: Script de instalação

## Métricas de Performance

- **Accuracy**: {accuracy:.3f}
- **Tempo de Inferência**: {avg_inference_time:.2f} ms
- **Throughput**: {1000/avg_inference_time:.1f} predições/segundo
- **Uso de Memória**: < 100 MB

## Monitoramento em Tempo Real

O sistema pode processar dados de rede em tempo real e:
- Detectar ataques com alta precisão
- Registrar detecções em arquivo JSON
- Mostrar estatísticas em tempo real
- Funcionar 24/7 em Raspberry Pi

## Integração com Sistemas de Rede

Para integrar com sistemas reais de monitoramento de rede:
1. Capture pacotes com ferramentas como tcpdump/Wireshark
2. Extraia features de rede (duração, flags, protocolos, etc.)
3. Alimente o detector em tempo real
4. Configure alertas para ataques detectados

## Suporte

Para dúvidas ou problemas, verifique:
- Logs do sistema em `attack_log.json`
- Compatibilidade das features de entrada
- Versões das dependências
'''

with open('README.md', 'w', encoding='utf-8') as f:
    f.write(readme)

print("Documentação criada!")

## 10. Download dos Arquivos para Raspberry Pi

In [ ]:
if IN_COLAB and df is not None:
    print("Preparando arquivos para download...")
    
    # Lista de arquivos para download
    files_to_download = [
        'network_attack_detector_quantized.onnx',
        'model_metadata.pkl',
        'realtime_network_monitor.py',
        'requirements.txt',
        'install.sh',
        'README.md'
    ]
    
    # Download dos arquivos
    for file in files_to_download:
        try:
            files.download(file)
            print(f"✅ {file}")
        except Exception as e:
            print(f"❌ Erro ao baixar {file}: {e}")
    
    print("\n=== RESUMO DO SISTEMA ===")
    print(f"📊 Accuracy do modelo: {accuracy:.3f}")
    print(f"⚡ Tempo de inferência: {avg_inference_time:.2f} ms")
    print(f"🚀 Throughput: {1000/avg_inference_time:.1f} predições/segundo")
    print(f"💾 Tamanho do modelo: {quantized_size:.1f} MB")
    print(f"🎯 Classes detectáveis: {len(label_encoder.classes_)}")
    
    print("\n=== PRÓXIMOS PASSOS ===")
    print("1. Transfira todos os arquivos para o Raspberry Pi")
    print("2. Execute: chmod +x install.sh && ./install.sh")
    print("3. Teste: python3 realtime_network_monitor.py --benchmark")
    print("4. Use com seus dados: python3 realtime_network_monitor.py --simulate seu_arquivo.csv")
    
elif df is not None:
    print("\nArquivos salvos localmente:")
    print("- network_attack_detector_quantized.onnx")
    print("- model_metadata.pkl")
    print("- realtime_network_monitor.py")
    print("- requirements.txt")
    print("- install.sh")
    print("- README.md")
else:
    print("⚠️ Nenhum dado foi carregado. Faça upload dos arquivos CSV primeiro.")